In [1]:
import pandas as pd
StockInfoTail = pd.read_hdf("D:/zxjt/StockInfoTail.h5")
IndexMinInfo = pd.read_hdf("D:/zxjt/IndexMinInfo.h5")
IndexConstituent = pd.read_hdf("D:/zxjt/IndexConstituent.h5")

In [2]:
StockInfoTail

,date,code,open,high,low,volume,amount,cjbs,close,vwap,limitedup,limiteddown,adj_factor,industry,listed_days,is_st,future_1d_ret,float_shares,free_shares
0,2015-01-05,000001.SZ,15.99,16.28,15.60,282760776,4.512796e+09,91928,16.01,15.959768,17.42,14.26,37.379405,银行,8769,False,-0.014981,983671.2150,496108.8120
1,2015-01-05,000002.SZ,14.39,15.29,14.22,652302247,9.644332e+09,184897,14.90,14.785067,15.29,12.51,89.647695,房地产,8742,False,-0.036888,967191.3090,802641.8370
2,2015-01-05,000004.SZ,15.58,15.90,15.30,2770612,4.313622e+07,2618,15.69,15.569204,17.13,14.01,3.008511,医药,8769,False,0.049076,8387.5459,6196.0792
3,2015-01-05,000006.SZ,7.10,7.39,7.02,106794512,7.677694e+08,24846,7.08,7.189222,7.76,6.35,59.090735,房地产,8288,False,-0.032486,133889.7143,78957.0969
4,2015-01-05,000007.SZ,14.51,14.80,14.25,2795472,4.056319e+07,2092,14.52,14.510319,16.06,13.14,8.156554,有色金属,8302,False,-0.017218,20581.3141,16202.0203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4600485,2020-12-31,688699.SH,54.00,56.33,53.70,1903941,1.055679e+08,3664,55.34,55.447029,65.04,43.36,1.000000,电子,13,False,0.019576,1691.4309,1691.4309
4600486,2020-12-31,688777.SH,99.58,102.53,98.65,2128937,2.141035e+08,4833,100.20,100.568252,119.98,79.98,1.000000,计算机,37,False,0.048104,3903.2884,3903.2884
4600487,2020-12-31,688788.SH,134.49,138.50,131.27,1302766,1.750315e+08,4299,133.50,134.353763,159.60,106.40,1.000000,国防军工,70,False,0.071118,1663.1354,1663.1354
4600488,2020-12-31,688981.SH,57.00,58.57,56.42,46555427,2.687157e+09,54617,57.60,57.719529,67.76,45.18,1.000000,电子,168,False,0.008831,104023.1223,104023.1223


In [3]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
import pickle
from gplearn import genetic
from gplearn.functions import make_function
from gplearn.genetic import SymbolicTransformer,SymbolicRegressor
from gplearn.fitness import make_fitness
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split


data = pd.read_hdf("D:/zxjt/StockInfoTail.h5")
data = data[~data.is_st]

tempt_code = list(pd.read_excel('D:/中证成分.xls').iloc[:,4].values)
index = data.pivot('code','date','open').index
final_code = []
for i in range(len(index)):
    tempt = int(index[i][:-3])
    if tempt in tempt_code:
        final_code.append(index[i])
        
        


open_data = data.pivot('code','date','open').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
close_data = data.pivot('code','date','close').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
high_data = data.pivot('code','date','high').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
low_data = data.pivot('code','date','low').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
volume_data = data.pivot('code','date','low').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
amount_data = data.pivot('code','date','amount').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
cjbs_data = data.pivot('code','date','cjbs').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
vwap_data = data.pivot('code','date','vwap').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
limitedup_data = data.pivot('code','date','limitedup').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
limiteddown_data = data.pivot('code','date','limiteddown').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
adj_factor_data = data.pivot('code','date','adj_factor').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
listed_days_data = data.pivot('code','date','listed_days').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
future_1d_ret_data = data.pivot('code','date','future_1d_ret').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
float_shares_data = data.pivot('code','date','float_shares').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
free_shares_data = data.pivot('code','date','free_shares').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]

C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2113745624.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  index = data.pivot('code','date','open').index
C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2113745624.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  open_data = data.pivot('code','date','open').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2113745624.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  close_data = data.pivot('code','date','close').apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))).fillna(0).loc[final_code,:]
C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2113745624.py:29: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  high_data =

In [4]:
date_list = close_data.columns[(close_data.columns>='2018-01-01' )]
train_date = date_list[date_list<='2019-12-31']
test_date = date_list[date_list>'2019-12-31']

In [5]:
rtn = data.pivot('code','date','future_1d_ret').fillna(0).loc[final_code,:]

C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2239896219.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  rtn = data.pivot('code','date','future_1d_ret').fillna(0).loc[final_code,:]


In [6]:
rtn1 = data.pivot('code','date','close').T.pct_change().shift(1).T.fillna(0).loc[final_code,:]
rtn3 = data.pivot('code','date','close').T.pct_change().shift(3).T.fillna(0).loc[final_code,:]
rtn5 = data.pivot('code','date','close').T.pct_change().shift(5).T.fillna(0).loc[final_code,:]
rtn10 = data.pivot('code','date','close').T.pct_change().shift(10).T.fillna(0).loc[final_code,:]
rtn30 = data.pivot('code','date','close').T.pct_change().shift(30).T.fillna(0).loc[final_code,:]

C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2192829071.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  rtn1 = data.pivot('code','date','close').T.pct_change().shift(1).T.fillna(0).loc[final_code,:]
C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2192829071.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  rtn3 = data.pivot('code','date','close').T.pct_change().shift(3).T.fillna(0).loc[final_code,:]
C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2192829071.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  rtn5 = data.pivot('code','date','close').T.pct_change().shift(5).T.fillna(0).loc[final_code,:]
C:\Users\deIl\AppData\Local\Temp\ipykernel_21656\2192829071.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  rtn10 = data.pivot('code','date','close').T

In [7]:
rtn1_train,rtn1_test = rtn1[train_date],rtn1[test_date]
rtn3_train,rtn3_test = rtn3[train_date],rtn3[test_date]
rtn5_train,rtn5_test = rtn5[train_date],rtn5[test_date]
rtn10_train,rtn10_test = rtn10[train_date],rtn10[test_date]
rtn30_train,rtn30_test = rtn30[train_date],rtn30[test_date]

In [8]:
rtn_train,rtn_test = rtn[train_date],rtn[test_date]

In [9]:
open_data_train,open_data_test = open_data[train_date],open_data[test_date]
high_data_train,high_data_test = high_data[train_date],high_data[test_date]
low_data_train,low_data_test = low_data[train_date],low_data[test_date]
close_data_train,close_data_test = close_data[train_date],close_data[test_date]
volume_data_train,volume_data_test = volume_data[train_date],volume_data[test_date]
amount_data_train,amount_data_test = amount_data[train_date],amount_data[test_date]
cjbs_data_train,cjbs_data_test = cjbs_data[train_date],cjbs_data[test_date]
vwap_data_train,vwap_data_test = vwap_data[train_date],vwap_data[test_date]
limitedup_data_train,limitedup_data_test = limitedup_data[train_date],limitedup_data[test_date]
limiteddown_data_train,limiteddown_data_test = limiteddown_data[train_date],limiteddown_data[test_date]
adj_factor_data_train,adj_factor_data_test = adj_factor_data[train_date],adj_factor_data[test_date]
listed_days_data_train,listed_days_data_test = listed_days_data[train_date],listed_days_data[test_date]
future_1d_ret_data_train,future_1d_ret_data_test = future_1d_ret_data[train_date],future_1d_ret_data[test_date]
float_shares_data_train,float_shares_data_test = float_shares_data[train_date],float_shares_data[test_date]
free_shares_data_train,free_shares_data_test = free_shares_data[train_date],free_shares_data[test_date]


In [10]:
data_train = np.concatenate([
        np.expand_dims(open_data_train.fillna(0), axis=2), 
        np.expand_dims(high_data_train.fillna(0), axis=2), 
        np.expand_dims(low_data_train.fillna(0), axis=2), 
        np.expand_dims(close_data_train.fillna(0), axis=2), 
        np.expand_dims(volume_data_train.fillna(0), axis=2), 
        np.expand_dims(amount_data_train.fillna(0), axis=2), 
        np.expand_dims(cjbs_data_train.fillna(0), axis=2),
        np.expand_dims(vwap_data_train.fillna(0), axis=2),
        np.expand_dims(limitedup_data_train.fillna(0), axis=2), 
        np.expand_dims(limiteddown_data_train.fillna(0), axis=2), 
        np.expand_dims(adj_factor_data_train.fillna(0), axis=2),
        np.expand_dims(listed_days_data_train.fillna(0), axis=2),
        #np.expand_dims(future_1d_ret_data_train.fillna(0), axis=2),
        np.expand_dims(float_shares_data_train.fillna(0), axis=2), 
        np.expand_dims(free_shares_data_train.fillna(0), axis=2), 
    ], axis=2)

data_test = np.concatenate([
        np.expand_dims(open_data_test.fillna(0), axis=2), 
        np.expand_dims(high_data_test.fillna(0), axis=2), 
        np.expand_dims(low_data_test.fillna(0), axis=2), 
        np.expand_dims(close_data_test.fillna(0), axis=2), 
        np.expand_dims(volume_data_test.fillna(0), axis=2), 
        np.expand_dims(amount_data_test.fillna(0), axis=2), 
        np.expand_dims(cjbs_data_test.fillna(0), axis=2),
        np.expand_dims(vwap_data_test.fillna(0), axis=2),
        np.expand_dims(limitedup_data_test.fillna(0), axis=2), 
        np.expand_dims(limiteddown_data_test.fillna(0), axis=2), 
        np.expand_dims(adj_factor_data_test.fillna(0), axis=2),
        np.expand_dims(listed_days_data_test.fillna(0), axis=2),
        #np.expand_dims(future_1d_ret_data_test.fillna(0), axis=2),
        np.expand_dims(float_shares_data_test.fillna(0), axis=2), 
        np.expand_dims(free_shares_data_test.fillna(0), axis=2), 
    ], axis=2)

In [11]:
import new_genetic as genetic
import new_program
import new_functions
import new_utils
import new_fitness
import pandas as pd
import numpy as np
import time
# feature_names = ['open','high','low','close','volume','amount','cjbs','vwap','limitedup',
#                  'limiteddown','adj_factor','listed_days','future_1d_ret','float_shares','free_shares']
feature_names = ['open','high','low','close','volume','amount','cjbs','vwap','limitedup',
                 'limiteddown','adj_factor','listed_days','float_shares','free_shares']
my_list_new = []
start = time.time()
for i in range(3):
    est_gp = genetic.SymbolicRegressor(
            population_size = 50, 
            generations = 10, 
            tournament_size = 50, 
            hall_of_fame = 50,
            init_depth = (1, 5), 
            stopping_criteria = 10, 
            max_samples = 0.8, 
            low_memory = True, 
            feature_names = feature_names, 
            #function_set = ['add','sub','mul','div'] ,
            metric = 'pearson1',
            #metric = fitness.make_fitness(IR, True), 
            verbose = 1,
            const_range = (3,30),
            init_method = 'half and half',
            n_jobs = 1,
            warm_start = True
        )

    est_gp.fit(X=data_train, y=rtn_train.values,X_test=data_test,y_test=rtn_test.values)
        
    program_list = []
    for i in est_gp._programs[-1]:
        #if i.raw_fitness_ >= 1:
        program_list.append(i)
    my_list1 = list(set([i.__str__() for i in program_list]))
    print(my_list1)
    my_list_new.extend(my_list1)

my_list_new = list(set(my_list_new))
pd.DataFrame(my_list_new).to_csv('D:/expression_final_new.csv')

end = time.time()

print(end-start)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness average_raw_fitness best_raw_fitness      OOB Fitness  Time Left
   0     8.52        0.0965672       39         0.380992          0.13807         0.421058         0.421058      7.45m
   1    23.98         0.197984       23          1.19234         0.231321          1.21599          1.21599      9.12m
   2    35.10          1.90505       55          71.3915          1.94212          71.4466          71.4466      9.25m
['delay(mul(mul(max(div(div(rank(vwap), ts_min(adj_factor, 20.000)), 27.000), ts_argmin(covariance(volume, cjbs, 14.000), 9.000)), 27.000), div(div(rank(vwap), ts_min(adj_factor, 20.000)), 27.000)), 20.000)', 'max(delay(ts_highday(div(ts_argmax(ts_argmin(amount, 29.000), 9.000), mul(mul(max(mul(low, 14.000), ts_argmin(covariance(volume, cjbs, 14.000), 9.000))

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness average_raw_fitness best_raw_fitness      OOB Fitness  Time Left
   0     8.22        0.0903084        9         0.289391         0.146961         0.305413         0.305413      7.76m
   1     8.20         0.162807        9         0.293287         0.182106         0.313442         0.313442     16.40m
   2     8.88         -8.03369        9         0.247534         0.198346         0.345977         0.345977     20.68m
   3     7.76        0.0695392        9         0.316806         0.225487         0.357468         0.357468     37.41m
   4    11.00         0.196441        9         0.317681         0.236081         0.351833         0.351833     14.71m
   5    10.10          0.20477        9         0.337729         0.225485         0.357468         0.357468     10.05m
   6

In [11]:
pd.DataFrame(my_list_new).to_csv('D:/expression_without_coef.csv')

In [10]:
my_list_new = list(set(my_list_new))
my_list_new

['ts_rank(amount, 5.000)',
 'ts_rank(volume, 17.000)',
 'delta(vwap, 7.000)',
 'ts_rank(close, 11.000)',
 'delta(close, 3.000)',
 'ts_argmax(limiteddown, 4.000)',
 'ts_argmax(vwap, 6.000)',
 'ts_rank(sub(sub(abs(ts_rank(close, 5.000)), log(signedpower(adj_factor))), correlation(ts_argmin(scale(listed_days), 14.000), mul(mul(volume, open), 25.000), 20.000)), 26.000)',
 'ts_rank(min(close, listed_days), 5.000)',
 'ts_rank(limitedup, 29.000)',
 'delta(open, 3.000)',
 'ts_rank(close, 4.000)',
 'ts_argmax(high, 3.000)',
 'delta(delta(close, 17.000), 4.000)',
 'rank(ts_rank(vwap, 8.000))',
 'ts_rank(high, 9.000)',
 'ts_highday(high, 5.000)',
 'ts_rank(close, 7.000)',
 'ts_rank(limiteddown, 28.000)',
 'ts_rank(volume, 26.000)',
 'delta(vwap, 10.000)',
 'ts_lowday(close, 7.000)',
 'ts_highday(high, 8.000)',
 'delta(rank(close), 6.000)',
 'ts_rank(low, 9.000)',
 'ts_rank(mul(low, 6.000), 22.000)',
 'ts_rank(rank(limitedup), 21.000)',
 'add(ts_rank(mul(abs(high), 15.000), 14.000), ts_wma(covaria

In [10]:
my_list_new = list(set(my_list_new))
pd.DataFrame(my_list_new).to_csv('D:/expression_new_new_2.csv')


In [23]:
my_list_new = list(set(my_list_new))
pd.DataFrame(my_list_new).to_csv('D:/expression_new_new_3.csv')

In [13]:
len(my_list_new)

165